In [1]:
import numpy as np 
import pandas as pd 

import os
print("Hello world")


Hello world


In [2]:
!git clone https://github.com/keras-rl/keras-rl.git

Cloning into 'keras-rl'...
remote: Enumerating objects: 1710, done.
remote: Total 1710 (delta 0), reused 0 (delta 0), pack-reused 1710
Receiving objects: 100% (1710/1710), 1.38 MiB | 10.00 MiB/s, done.
Resolving deltas: 100% (1062/1062), done.


In [3]:
!ls

keras-rl  sample_data


In [4]:
!git clone https://github.com/anant9798/GamePlaying.git

Cloning into 'GamePlaying'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 218 (delta 0), reused 0 (delta 0), pack-reused 215
Receiving objects: 100% (218/218), 66.03 MiB | 37.58 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [5]:
!python GamePlaying/Pong-DQN/keras-rl-lib/examples/dqn_cartpole.py

Using TensorFlow backend.
Traceback (most recent call last):
  File "GamePlaying/Pong-DQN/keras-rl-lib/examples/dqn_cartpole.py", line 8, in <module>
    from rl.agents.dqn import DQNAgent
ModuleNotFoundError: No module named 'rl'


In [6]:
!cd keras-rl/examples/ && ls

cem_cartpole.py   dqn_atari.py		naf_pendulum.py
ddpg_mujoco.py	  dqn_cartpole.py	sarsa_cartpole.py
ddpg_pendulum.py  duel_dqn_cartpole.py	visualize_log.py


In [0]:
!cp -R keras-rl/rl/ .

In [8]:
!ls

GamePlaying  keras-rl  rl  sample_data


In [12]:
!cd rl/ && ls

agents	      common   __init__.py  policy.py	   __pycache__	util.py
callbacks.py  core.py  memory.py    processors.py  random.py


In [19]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


ENV_NAME = 'CartPole-v1'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=150, visualize=False)
print("Testing done")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_34 (Dense)             (None, 16)                80        
_________________________________________________________________
activation_32 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_33 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_34 (Activation)   (None, 16)                0         
__________

/content/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    31/50000: episode: 1, duration: 1.729s, episode steps: 31, steps per second: 18, episode reward: 31.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.419 [0.000, 1.000], mean observation: 0.013 [-1.185, 1.776], loss: 0.462173, mean_absolute_error: 0.520752, mean_q: 0.096439
    44/50000: episode: 2, duration: 0.084s, episode steps: 13, steps per second: 154, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.692 [0.000, 1.000], mean observation: -0.069 [-1.867, 1.224], loss: 0.352699, mean_absolute_error: 0.544416, mean_q: 0.290915


/content/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    64/50000: episode: 3, duration: 0.142s, episode steps: 20, steps per second: 141, episode reward: 20.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.550 [0.000, 1.000], mean observation: -0.070 [-1.375, 0.833], loss: 0.233226, mean_absolute_error: 0.553622, mean_q: 0.481963
    83/50000: episode: 4, duration: 0.120s, episode steps: 19, steps per second: 158, episode reward: 19.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.421 [0.000, 1.000], mean observation: 0.074 [-0.817, 1.505], loss: 0.115771, mean_absolute_error: 0.604906, mean_q: 0.841101
    98/50000: episode: 5, duration: 0.099s, episode steps: 15, steps per second: 151, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: -0.090 [-1.368, 0.825], loss: 0.060949, mean_absolute_error: 0.689512, mean_q: 1.187350
   114/50000: episode: 6, duration: 0.103s, episode steps: 16, steps per second: 156, episode reward: 16.000, mean reward: 1.000 [1.000, 1.000]

In [20]:
!ls

dqn_CartPole-v1_weights.h5f	  GamePlaying  rl
duel_dqn_CartPole-v1_weights.h5f  keras-rl     sample_data


In [21]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


ENV_NAME = 'CartPole-v1'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model regardless of the dueling architecture
# if you enable dueling network in DQN , DQN will build a dueling network base on your model automatically
# Also, you can build a dueling network by yourself and turn off the dueling network in DQN.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions, activation='linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
# enable the dueling network
# you can specify the dueling_type to one of {'avg','max','naive'}
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

# After training is done, we save the final weights.
dqn.save_weights('duel_dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=150, visualize=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 4)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 16)                80        
_________________________________________________________________
activation_36 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_37 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_38 (Activation)   (None, 16)                0         
__________

/content/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    20/50000: episode: 1, duration: 2.144s, episode steps: 20, steps per second: 9, episode reward: 20.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.400 [0.000, 1.000], mean observation: 0.059 [-0.998, 1.652], loss: 0.467027, mean_absolute_error: 0.514025, mean_q: -0.058345


/content/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    73/50000: episode: 2, duration: 0.366s, episode steps: 53, steps per second: 145, episode reward: 53.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.566 [0.000, 1.000], mean observation: 0.218 [-1.248, 1.743], loss: 0.341551, mean_absolute_error: 0.471468, mean_q: 0.132281
   106/50000: episode: 3, duration: 0.237s, episode steps: 33, steps per second: 139, episode reward: 33.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.515 [0.000, 1.000], mean observation: -0.029 [-1.201, 0.833], loss: 0.125812, mean_absolute_error: 0.516086, mean_q: 0.589520
   116/50000: episode: 4, duration: 0.072s, episode steps: 10, steps per second: 140, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.900 [0.000, 1.000], mean observation: -0.152 [-2.770, 1.738], loss: 0.033075, mean_absolute_error: 0.600775, mean_q: 0.999000
   139/50000: episode: 5, duration: 0.163s, episode steps: 23, steps per second: 141, episode reward: 23.000, mean reward: 1.000 [1.000, 1.000]

In [22]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy


ENV_NAME = 'CartPole-v1'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=50000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=150, visualize=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 4)                 0         
_________________________________________________________________
dense_43 (Dense)             (None, 16)                80        
_________________________________________________________________
activation_39 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_40 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_41 (Activation)   (None, 16)                0         
__________

In [23]:
!ls

dqn_CartPole-v1_weights.h5f	  keras-rl     sarsa_CartPole-v1_weights.h5f
duel_dqn_CartPole-v1_weights.h5f  rl
GamePlaying			  sample_data
